<a href="https://colab.research.google.com/github/chaerui7967/stock_predict_news_and_youtube/blob/master/lg%ED%99%94%ED%95%99_%ED%86%B5%ED%95%A9%ED%85%8C%EC%9D%B4%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 오늘 시작

## 라이브러리 설치

In [1]:
!pip install finance-datareader
!pip install pykrx
!pip install pymysql
!pip install konlpy
from konlpy.tag import Okt
okt = Okt()

     |████████████████████████████████| 81 kB 6.2 MB/s 
     |████████████████████████████████| 60 kB 7.4 MB/s 
     |████████████████████████████████| 251 kB 40.2 MB/s 
     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 19.4 MB 5.1 MB/s 
     |████████████████████████████████| 86 kB 6.1 MB/s 
     |████████████████████████████████| 448 kB 43.6 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


## 패키지 & 함수 로딩

In [2]:
# 필요 라이브러리
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 주식가격 로딩
import FinanceDataReader as fdr
from pykrx import stock

import datetime

In [3]:
def money_sur(stock_name, start, end):

    start_date = start
    end_date = end
    sample_code = stock_name

    dataset = fdr.DataReader(sample_code, start = start_date, end = end_date )
    dataset = dataset.reset_index()
    df = stock.get_market_fundamental_by_date(fromdate=start_date, todate=end_date, ticker=sample_code)
    df = df.reset_index()
    df.columns = ['Date','BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS']
    data = pd.merge(dataset, df, on='Date')

    return data

In [4]:
def reverse_min_max_scaling(org_x, x): #종가 예측값
    org_x_np = np.asarray(org_x) 
    x_np = np.asarray(x)
    return (x_np * (org_x_np.max() - org_x_np.min())) + org_x_np.min()

## 통합테이블 생성

In [5]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# total_table = pd.read_csv('/content/drive/My Drive/Final PJT - 업빛투/total_table.csv') # 기존 테이블

In [7]:
# 구성
dic = {'code' : [],
       'date' : [],
       'lstm' : [],
       'arima' : [],
       'fbprophet' : [],
       'RL' : [],
    #    'mail_news' : [],
    #    'mail_news_nsi' : [],
    #    'asia_news' : [],
    #    'asia_news_nsi': [],
    #    'youtube_sam' : [],
    #    'youtube_su' : [],
    #    'youtube_han' : [],
       'close' : []}

total_table = pd.DataFrame(dic)

In [8]:
total_table.columns

Index(['code', 'date', 'lstm', 'arima', 'fbprophet', 'RL', 'close'], dtype='object')

In [9]:
total_table

,code,date,lstm,arima,fbprophet,RL,close


## LSTM 예측결과

### 사용 데이터 로드 및 전처리

In [10]:
start_date = (datetime.datetime.now() - datetime.timedelta(40)).strftime("%Y%m%d")
end_date = datetime.datetime.now().strftime("%Y%m%d")
lstm = money_sur('051910', start_date, end_date)

In [11]:
lstm = lstm.set_index('Date')

In [12]:
lstm.head()

,Open,High,Low,Close,Volume,Change,BPS,PER,PBR,EPS,DIV,DPS
Date,,,,,,,,,,,,
2021-08-23,865000,869000,789000,798000,944586,-0.111359,230440,119.71,3.46,6666,1.25,10000
2021-08-24,775000,810000,772000,787000,895863,-0.013784,230440,118.06,3.42,6666,1.27,10000
2021-08-25,791000,805000,790000,799000,459360,0.015248,230440,119.86,3.47,6666,1.25,10000
2021-08-26,798000,802000,785000,788000,240562,-0.013767,230440,118.21,3.42,6666,1.27,10000
2021-08-27,782000,788000,757000,783000,353049,-0.006345,230440,117.46,3.40,6666,1.28,10000


In [13]:
# 스케일링
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler()
lstm_sc = sc.fit_transform(lstm)

In [14]:
scale_cols = lstm.columns
lstm_sc_df = pd.DataFrame(lstm_sc, columns=[scale_cols], index=lstm.index)

In [15]:
lstm_sc_df.head()

,Open,High,Low,Close,Volume,Change,BPS,PER,PBR,EPS,DIV,DPS
Date,,,,,,,,,,,,
2021-08-23,1.000000,1.000000,0.989362,0.989796,1.000000,0.000000,0.0,0.989796,0.976744,0.0,0.000000,0.0
2021-08-24,0.444444,0.621795,0.808511,0.877551,0.931744,0.499039,0.0,0.877551,0.883721,0.0,0.111111,0.0
2021-08-25,0.543210,0.589744,1.000000,1.000000,0.320243,0.647523,0.0,1.000000,1.000000,0.0,0.000000,0.0
2021-08-26,0.586420,0.570513,0.946809,0.887755,0.013727,0.499127,0.0,0.887755,0.883721,0.0,0.111111,0.0
2021-08-27,0.487654,0.480769,0.648936,0.836735,0.171311,0.537087,0.0,0.836735,0.837209,0.0,0.166667,0.0


In [16]:
for i in scale_cols:
  for s in range(1, 11):  
    lstm_sc_df[(i+'shift_{}'.format(s),)] = lstm_sc_df[(i,)].shift(s)

In [17]:
lstm_sc_df.dropna()

,Open,High,Low,Close,Volume,Change,BPS,PER,PBR,EPS,DIV,DPS,Openshift_1,Openshift_2,Openshift_3,Openshift_4,Openshift_5,Openshift_6,Openshift_7,Openshift_8,Openshift_9,Openshift_10,Highshift_1,Highshift_2,Highshift_3,Highshift_4,Highshift_5,Highshift_6,Highshift_7,Highshift_8,Highshift_9,Highshift_10,Lowshift_1,Lowshift_2,Lowshift_3,Lowshift_4,Lowshift_5,Lowshift_6,Lowshift_7,Lowshift_8,...,PBRshift_1,PBRshift_2,PBRshift_3,PBRshift_4,PBRshift_5,PBRshift_6,PBRshift_7,PBRshift_8,PBRshift_9,PBRshift_10,EPSshift_1,EPSshift_2,EPSshift_3,EPSshift_4,EPSshift_5,EPSshift_6,EPSshift_7,EPSshift_8,EPSshift_9,EPSshift_10,DIVshift_1,DIVshift_2,DIVshift_3,DIVshift_4,DIVshift_5,DIVshift_6,DIVshift_7,DIVshift_8,DIVshift_9,DIVshift_10,DPSshift_1,DPSshift_2,DPSshift_3,DPSshift_4,DPSshift_5,DPSshift_6,DPSshift_7,DPSshift_8,DPSshift_9,DPSshift_10
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-09-06,0.104938,0.262821,0.223404,0.500000,0.441551,0.745900,0.0,0.500000,0.488372,0.0,0.444445,0.0,0.000000,0.074074,0.370370,0.413580,0.518519,0.487654,0.586420,0.543210,0.444444,1.000000,0.083333,0.051282,0.339744,0.410256,0.487179,0.480769,0.570513,0.589744,0.621795,1.000000,0.000000,0.095745,0.265957,0.595745,0.755319,0.648936,0.946809,1.000000,...,0.255814,0.116279,0.209303,0.581395,0.697674,0.837209,0.883721,1.000000,0.883721,0.976744,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.722222,0.888889,0.777778,0.388889,0.277778,0.166667,0.111111,0.000000,0.111111,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-07,0.333333,0.307692,0.542553,0.581633,0.253667,0.624093,0.0,0.581633,0.581395,0.0,0.388889,0.0,0.104938,0.000000,0.074074,0.370370,0.413580,0.518519,0.487654,0.586420,0.543210,0.444444,0.262821,0.083333,0.051282,0.339744,0.410256,0.487179,0.480769,0.570513,0.589744,0.621795,0.223404,0.000000,0.095745,0.265957,0.595745,0.755319,0.648936,0.946809,...,0.488372,0.255814,0.116279,0.209303,0.581395,0.697674,0.837209,0.883721,1.000000,0.883721,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.444445,0.722222,0.888889,0.777778,0.388889,0.277778,0.166667,0.111111,0.000000,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-08,0.339506,0.301282,0.563830,0.561224,0.099166,0.556044,0.0,0.561224,0.558139,0.0,0.388889,0.0,0.333333,0.104938,0.000000,0.074074,0.370370,0.413580,0.518519,0.487654,0.586420,0.543210,0.307692,0.262821,0.083333,0.051282,0.339744,0.410256,0.487179,0.480769,0.570513,0.589744,0.542553,0.223404,0.000000,0.095745,0.265957,0.595745,0.755319,0.648936,...,0.581395,0.488372,0.255814,0.116279,0.209303,0.581395,0.697674,0.837209,0.883721,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.388889,0.444445,0.722222,0.888889,0.777778,0.388889,0.277778,0.166667,0.111111,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-09,0.277778,0.250000,0.457447,0.479592,0.116392,0.515418,0.0,0.479592,0.488372,0.0,0.500000,0.0,0.339506,0.333333,0.104938,0.000000,0.074074,0.370370,0.413580,0.518519,0.487654,0.586420,0.301282,0.307692,0.262821,0.083333,0.051282,0.339744,0.410256,0.487179,0.480769,0.570513,0.563830,0.542553,0.223404,0.000000,0.095745,0.265957,0.595745,0.755319,...,0.558139,0.581395,0.488372,0.255814,0.116279,0.209303,0.581395,0.697674,0.837209,0.883721,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.388889,0.388889,0.444445,0.722222,0.888889,0.777778,0.388889,0.277778,0.166667,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-10,0.277778,0.256410,0.468085,0.397959,0.051583,0.514839,0.0,0.397959,0.395349,0.0,0.555556,0.0,0.277778,0.339506,0.333333,0.104938,0.000000,0.074074,0.370370,0.413580,0.518519,0.487654,0.250000,0.301282,0.307692,0.262821,0.083333,0.051282,0.339744,0.410256,0.487179,0.480769,0.457447,0.563830,0.542553,0.223404,0.000000,0.095745,0.265957,0.595745,...,0.488372,0.558139,0.581395,0.488372,0.255814,0.116279,0.209303,0.581395,0.697674,0.837209,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.388889,0.388889,0.444445,0.722222,0.888889,0.777778,0.388889,0.277778,0.166667,0.0,0.0,0.0,0.0,0

In [18]:
X_test = lstm_sc_df.dropna().drop(columns=scale_cols, axis=1)
y_test = lstm_sc_df.dropna()[['Close']]

X_test= X_test.values
y_test = y_test.values

In [19]:
X_test

array([[0.        , 0.07407407, 0.37037037, ..., 0.        , 0.        ,
        0.        ],
       [0.10493827, 0.        , 0.07407407, ..., 0.        , 0.        ,
        0.        ],
       [0.33333333, 0.10493827, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.45679012, 0.37037037, 0.41358025, ..., 0.        , 0.        ,
        0.        ],
       [0.36419753, 0.45679012, 0.37037037, ..., 0.        , 0.        ,
        0.        ],
       [0.31481481, 0.36419753, 0.45679012, ..., 0.        , 0.        ,
        0.        ]])

In [20]:
X_test_t = X_test.reshape(X_test.shape[0], 10, 12)

### 모델 로드

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Lambda,Dropout,Flatten
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()

model.add(LSTM(64,
               input_shape = (10, 12),
               return_sequences= True))
model.add(Dropout(0.2))
model.add(Dense(32, activation = 'relu'))
model.add(LSTM(64,
               input_shape = (10, 12),
               return_sequences= True))
model.add(Dropout(0.2))
model.add(Dense(32, activation = 'tanh'))
model.add(LSTM(64,
               return_sequences = False))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(100, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(1))



lstm_path = '/content/drive/My Drive/Final PJT - 업빛투/Regression/LSTM/'
lg_lstm = 'lstm_lg.h5'

model.load_weights(lstm_path+lg_lstm)

### 모델 결과

In [22]:
pred = model.predict(X_test_t)

In [23]:
pred.shape, y_test.shape

((17, 1), (17, 1))

In [24]:
pred_act = reverse_min_max_scaling(lstm['Close'], pred)

In [25]:
total_table['date'] = lstm.index[-10:]
total_table['lstm'] = pred_act[-10:]
total_table['close'] = lstm.Close[-10:].values.reshape(-1,1)
total_table['code'] = '051910'
total_table['change'] = lstm.Change[-10:].values.reshape(-1,1)

In [26]:
total_table

,code,date,lstm,arima,fbprophet,RL,close,change
0,051910,2021-09-15,733916.571140,NaN,NaN,NaN,718000,-0.021798
1,051910,2021-09-16,730745.079160,NaN,NaN,NaN,716000,-0.002786
2,051910,2021-09-17,727474.198222,NaN,NaN,NaN,701000,-0.020950
3,051910,2021-09-23,723239.910305,NaN,NaN,NaN,760000,0.084165
4,051910,2021-09-24,721332.248360,NaN,NaN,NaN,761000,0.001316
5,051910,2021-09-27,721468.415320,NaN,NaN,NaN,770000,0.011827
6,051910,2021-09-28,725266.111761,NaN,NaN,NaN,776000,0.007792
7,051910,2021-09-29,730308.284700,NaN,NaN,NaN,767000,-0.011598
8,051910,2021-09-30,732722.755849,NaN,NaN,NaN,776000,0.011734
9,051910,2021-10-01,736871.808171,NaN,NaN,NaN,770000,-0.007732


## arima 예측결과

In [30]:
end_date = datetime.datetime.now().strftime("%Y%m%d")
samsung_df = money_sur('051910', '2018', end_date)

In [31]:
df = samsung_df[['Date','Close']]
df.columns = ['Date', 'Price']

In [32]:
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
# 학습, 테스트 데이터 분리
for ru in range(-10,0):
    rus = ru+1
    train = df[:ru]
    test = df[ru:]

    model = ARIMA(train.Price.values, order = (2,2,0))
    model_fit = model.fit(trend = 'nc', full_output = True, disp = True)

    fore = model_fit.forecast(steps=1) # 오늘 예측

    # 오늘 예측 데이터
    pred_arima_y = fore[0].tolist()

    # 실제 오늘 데이터 
    test_y = test.Price.values

    # # 신뢰구간
    # # 예측 데이터 최소값
    # pred_y_lower = []
    # # 예측 데이터 최대값
    # pred_y_upper = []

    # for lower_upper in fore[2]:
    #     lower = lower_upper[0]
    #     upper = lower_upper[1]
    #     pred_y_lower.append(lower)
    #     pred_y_upper.append(upper)
    if rus == 0:
        total_table['arima'].iloc[ru:] = pred_arima_y
    else:
        total_table['arima'].iloc[ru:rus] = pred_arima_y

print(model_fit.summary())

                             ARIMA Model Results                              
Dep. Variable:                   D2.y   No. Observations:                  921
Model:                 ARIMA(2, 2, 0)   Log Likelihood              -10262.489
Method:                       css-mle   S.D. of innovations          16705.698
Date:                Sat, 02 Oct 2021   AIC                          20530.978
Time:                        08:06:37   BIC                          20545.455
Sample:                             2   HQIC                         20536.502
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1.D2.y    -0.7008      0.031    -22.897      0.000      -0.761      -0.641
ar.L2.D2.y    -0.3693      0.031    -12.066      0.000      -0.429      -0.309
                                    Roots           

In [33]:
total_table

,code,date,lstm,arima,fbprophet,RL,close,change
0,051910,2021-09-15,733916.571140,729389.675560,NaN,NaN,718000,-0.021798
1,051910,2021-09-16,730745.079160,711531.412953,NaN,NaN,716000,-0.002786
2,051910,2021-09-17,727474.198222,705344.846383,NaN,NaN,701000,-0.020950
3,051910,2021-09-23,723239.910305,689864.156645,NaN,NaN,760000,0.084165
4,051910,2021-09-24,721332.248360,772048.242011,NaN,NaN,761000,0.001316
5,051910,2021-09-27,721468.415320,775235.263126,NaN,NaN,770000,0.011827
6,051910,2021-09-28,725266.111761,794974.525572,NaN,NaN,776000,0.007792
7,051910,2021-09-29,730308.284700,781150.824165,NaN,NaN,767000,-0.011598
8,051910,2021-09-30,732722.755849,769613.268728,NaN,NaN,776000,0.011734
9,051910,2021-10-01,736871.808171,777926.111086,NaN,NaN,770000,-0.007732


## fbprophet 예측

In [34]:
end_date = datetime.datetime.now().strftime("%Y%m%d")
samsung_df = money_sur('051910', start = '20180101', end = end_date)
df1 = samsung_df[['Date','Close']]
df1.columns = ['ds', 'y']

from fbprophet import Prophet

prophet = Prophet(seasonality_mode = 'multiplicative',
                 yearly_seasonality=True, 
                 weekly_seasonality=True,
                 daily_seasonality=True,
                 changepoint_prior_scale=0.6)

prophet.fit(df1)

# 10일단위로 예측값을 가져옴
future_data = prophet.make_future_dataframe(periods = 10, freq = 'd')
forecast_data = prophet.predict(future_data)

In [35]:
list(total_table.date.values)

[numpy.datetime64('2021-09-15T00:00:00.000000000'),
 numpy.datetime64('2021-09-16T00:00:00.000000000'),
 numpy.datetime64('2021-09-17T00:00:00.000000000'),
 numpy.datetime64('2021-09-23T00:00:00.000000000'),
 numpy.datetime64('2021-09-24T00:00:00.000000000'),
 numpy.datetime64('2021-09-27T00:00:00.000000000'),
 numpy.datetime64('2021-09-28T00:00:00.000000000'),
 numpy.datetime64('2021-09-29T00:00:00.000000000'),
 numpy.datetime64('2021-09-30T00:00:00.000000000'),
 numpy.datetime64('2021-10-01T00:00:00.000000000')]

In [36]:
total_table['fbprophet'] = forecast_data[forecast_data['ds'].isin(list(total_table.date.values))]['yhat'].values

In [37]:
total_table

,code,date,lstm,arima,fbprophet,RL,close,change
0,051910,2021-09-15,733916.571140,729389.675560,750428.645077,NaN,718000,-0.021798
1,051910,2021-09-16,730745.079160,711531.412953,755533.388291,NaN,716000,-0.002786
2,051910,2021-09-17,727474.198222,705344.846383,755657.104610,NaN,701000,-0.020950
3,051910,2021-09-23,723239.910305,689864.156645,748824.721482,NaN,760000,0.084165
4,051910,2021-09-24,721332.248360,772048.242011,747809.020471,NaN,761000,0.001316
5,051910,2021-09-27,721468.415320,775235.263126,738985.498854,NaN,770000,0.011827
6,051910,2021-09-28,725266.111761,794974.525572,736767.834210,NaN,776000,0.007792
7,051910,2021-09-29,730308.284700,781150.824165,730421.099476,NaN,767000,-0.011598
8,051910,2021-09-30,732722.755849,769613.268728,733394.311545,NaN,776000,0.011734
9,051910,2021-10-01,736871.808171,777926.111086,731385.012952,NaN,770000,-0.007732


In [38]:
# total_table.to_csv('./total_table.csv', index =False)

## RL 결과

### 데이터 로딩 및 함수 로딩

In [39]:
import time
import copy
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F

# 금융정보 조회하기
def dataset_loaderKR(stock_name, start, end, train_ratio=0.8):
    start_date = start
    end_date = end
    sample_code = stock_name

    dataset = fdr.DataReader(sample_code, start = start_date, end = end_date )
    dataset = dataset.reset_index()
    df = stock.get_market_fundamental_by_date(fromdate=start_date, todate=end_date, ticker=sample_code)
    df = df.reset_index()
    df.columns = ['Date','BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS']
    data = pd.merge(dataset, df, on='Date')
    date_split = data.index[int(train_ratio*len(data))] 

    return data[:date_split], data[date_split:], date_split


# ENV 설정
class Environment1:
    def __init__(self, data, history_t=90):
        self.data = data
        self.history_t = history_t
        self.reset()
        
    def reset(self):
        self.t = 0
        self.done = False
        self.profits = 0
        self.positions = []
        self.position_value = 0
        self.history = [0 for _ in range(self.history_t)]
        return [self.position_value] + self.history # obs

    def step(self, act):
        reward = 0
        
        # action
        # 0: Idle
        # 1: 매수
        # 2: 매도
        if act == 1: #매수
            self.positions.append(self.data.iloc[self.t, :]['Close'])
        elif act == 2: # 매도
            if len(self.positions) == 0:
                reward = -1
            else:
                profits = 0
                for p in self.positions:
                    profits += (self.data.iloc[self.t, :]['Close'] - p)
                reward += profits
                self.profits += profits
                self.positions = []
        
        # set next time
        self.t += 1
        
        self.position_value = 0
        for p in self.positions:
            self.position_value += (self.data.iloc[self.t, :]['Close'] - p)
        self.history.pop(0)
        self.history.append(self.data.iloc[self.t, :]['Close'] - self.data.iloc[(self.t-1), :]['Close'])
        if (self.t==len(self.data)-1):
            self.done=True
        # clipping reward
        if reward > 0:
            reward = 1
        elif reward < 0:
            reward = -1
        #print ("t={%d}, done={%str}"%(self.t,self.done))
        return [self.position_value] + self.history, reward, self.done # obs, reward, done

# MODEL
class Q_Network(nn.Module):
    def __init__(self,obs_len,hidden_size,actions_n):
        super(Q_Network,self).__init__()
        self.fc_val = nn.Sequential(
            nn.Linear(obs_len, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, actions_n)
        )
    def forward(self,x):
        h = self.fc_val(x)
        return (h)

In [40]:
(train, test, date_split) = dataset_loaderKR('051910', start = '20180101', end = end_date)

### 모델 로딩 및 결과

In [45]:
rl_path = '/content/drive/My Drive/Final PJT - 업빛투/Regression/RL/LG화학_결과/'
lg_rl = 'lg_rl_재무정보.h5'

In [46]:
# 모델 로딩
model = torch.load(rl_path+lg_rl)

test_env = Environment1(test)
pobs = test_env.reset()
test_acts = []
test_rewards = []

for _ in range(len(test_env.data)-1):
    pact = model(torch.from_numpy(np.array(pobs, dtype=np.float32).reshape(1, -1)))
    pact = np.argmax(pact.data)
    test_acts.append(pact.item())
            
    obs, reward, done = test_env.step(pact.numpy())
    test_rewards.append(reward)

    pobs = obs

# sell remaining stocks
profits = 0
for p in test_env.positions:
	profits += (test_env.data.iloc[test_env.t, :]['Close'] - p)

test_env.profits += profits
test_env.positions = []

test_profits = test_env.profits
pact = model(torch.from_numpy(np.array(pobs, dtype=np.float32).reshape(1, -1)))
pact = np.argmax(pact.data)
next_action = pact.item()
test['Action'] = test_acts + [next_action]


print( )
print( test )
print( )
print("결과")
print( "손익결과 $%.2f" % round(test_profits,2) )
# 0: Idle
# 1: 매수
# 2: 매도


          Date     Open     High     Low  ...   EPS   DIV    DPS  Action
739 2021-01-05   891000   893000  875000  ...  4085  0.22   2000       0
740 2021-01-06   908000   912000  883000  ...  4085  0.22   2000       2
741 2021-01-07   910000   969000  896000  ...  4085  0.21   2000       2
742 2021-01-08   989000  1016000  970000  ...  4085  0.20   2000       1
743 2021-01-11  1012000  1045000  981000  ...  4085  0.20   2000       2
..         ...      ...      ...     ...  ...   ...   ...    ...     ...
919 2021-09-27   763000   770000  757000  ...  6666  1.30  10000       2
920 2021-09-28   777000   783000  765000  ...  6666  1.29  10000       2
921 2021-09-29   762000   769000  749000  ...  6666  1.30  10000       2
922 2021-09-30   754000   780000  748000  ...  6666  1.29  10000       1
923 2021-10-01   778000   781000  764000  ...  6666  1.30  10000       2

[185 rows x 14 columns]

결과
손익결과 $-49000.00


In [47]:
test['Action'] = np.where(test['Action'] == 1,1,
                          np.where(test['Action'] == 2,-1,0))

In [48]:
test['Action'][-10:]

914    1
915    0
916   -1
917   -1
918   -1
919   -1
920   -1
921   -1
922    1
923   -1
Name: Action, dtype: int64

In [49]:
total_table['RL'] = test[test['Date'].isin(list(total_table.date.values))]['Action'].values

In [50]:
total_table

,code,date,lstm,arima,fbprophet,RL,close,change
0,051910,2021-09-15,733916.571140,729389.675560,750428.645077,1,718000,-0.021798
1,051910,2021-09-16,730745.079160,711531.412953,755533.388291,0,716000,-0.002786
2,051910,2021-09-17,727474.198222,705344.846383,755657.104610,-1,701000,-0.020950
3,051910,2021-09-23,723239.910305,689864.156645,748824.721482,-1,760000,0.084165
4,051910,2021-09-24,721332.248360,772048.242011,747809.020471,-1,761000,0.001316
5,051910,2021-09-27,721468.415320,775235.263126,738985.498854,-1,770000,0.011827
6,051910,2021-09-28,725266.111761,794974.525572,736767.834210,-1,776000,0.007792
7,051910,2021-09-29,730308.284700,781150.824165,730421.099476,-1,767000,-0.011598
8,051910,2021-09-30,732722.755849,769613.268728,733394.311545,1,776000,0.011734
9,051910,2021-10-01,736871.808171,777926.111086,731385.012952,-1,770000,-0.007732


## News 긍부정 결과
- '매일경제', '아시아경제'

In [51]:
# corp_list = ['samsung', 'hyundai', 'lg', 'sk', 'celltrion'] 참고용
stock_num_list = ['005930', '005380', '051910', '000660', '068270']
news = ['maeil_news_craw', 'asia_news_craw']
start = (datetime.datetime.now() - datetime.timedelta(20)).strftime("%Y-%m-%d").replace('-','')
end = datetime.datetime.now().strftime("%Y-%m-%d").replace('-', '')

In [52]:
# news_df = pd.read_csv('/content/drive/My Drive/Final PJT - 업빛투/감성분석/Final Data/1. samsung_score.csv') # 연습용

### 관련 함수 및 패키지 로드

In [53]:
import pymysql
import sys
import re

In [54]:
# 블로그에서 가져온 기본적인 한국어 긍부정 텍스트 목록
with open('/content/drive/My Drive/Final PJT - 업빛투/감성분석/data/positive_words_self.txt', encoding='utf-8') as pos_blog:
    positive_blog = pos_blog.readlines()
positive_blog = [pos_blog.replace('\n', '') for pos_blog in positive_blog]
with open('/content/drive/My Drive/Final PJT - 업빛투/감성분석/data/negative_words_self.txt', encoding='utf-8') as neg_blog:
    negative_blog = neg_blog.readlines()
negative_blog = [neg_blog.replace('\n', '') for neg_blog in negative_blog]

# KOSELF 감성 어휘 사전
with open('/content/drive/My Drive/Final PJT - 업빛투/감성분석/data/KOSELF_pos.txt', encoding='utf-8') as pos:
    positive = pos.readlines()
positive = [pos.replace('\n', '') for pos in positive]
with open('/content/drive/My Drive/Final PJT - 업빛투/감성분석/data/KOSELF_neg.txt', encoding='utf-8') as neg:
    negative = neg.readlines()
negative = [neg.replace('\n', '') for neg in negative]

In [55]:
def news_db(news, stock):
    db = pymysql.connect(
            user='root', 
            passwd='1234', 
            host='3.35.70.166', 
            db='proj', 
            charset = 'utf8'
        )

    cursor = db.cursor(pymysql.cursors.DictCursor)
    news1 = news + '_news_craw'

    sql = "select * from {0}_{1} where (length(date)=10) and (date between {2}00 and {3}23)".format(news1, stock, start, end)
    cursor.execute(sql)
    result = cursor.fetchall()

    # DataFrame으로 변경
    df = pd.DataFrame(result)

    db.close()

    df.rename(columns={'date': 'datetime'}, inplace=True)

    # DataFrame 형태를 통일하기 위해 date 컬럼 추가
    df['date'] = df['datetime'].str[0:4] + '-' + df['datetime'].str[4:6] + '-' + df['datetime'].str[6:8]
    df['date'] = pd.to_datetime(df['date'])

    # 결측치 제거 → 데이터 로드 시 완료했기 때문에 그다지 필요하지 않은 과정
    df = df.dropna()

    # 시간순으로 정렬
    df.sort_values('datetime', inplace=True)
    df.reset_index(inplace=True, drop=True)

    ### 4) Holidays
    db = pymysql.connect(user='root',
                         passwd='1234', 
                         host='3.35.70.166', 
                         db='proj',
                         charset='utf8')

    cursor = db.cursor(pymysql.cursors.DictCursor)

    # 4-1) 주말 및 공휴일 데이터
    sql = "select * from holidays"
    cursor.execute(sql)
    result = cursor.fetchall()

    holidays = pd.DataFrame(result)

    db.close()

    holidays['date'] = pd.to_datetime(holidays['date'])

    url = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/Sentiment_Analysis/data/stopwords_ver1.txt'
    stopwords = list(pd.read_csv(url, header=None)[0])

    # Stop Words List에 각 매체명 추가
    except_media_list = ['매일경제', '매일', '경제', 'maeil', 'MK', 'mk',
                         '아시아경제', '아시아', 'Asia', 'ASIA', 'asia',
                         '삼프로TV', '삼프로', 'TV',
                         '슈카월드', '슈카', '월드'
                         '한국경제TV', '한국']
    for word in except_media_list:
        stopwords.append(word)
    
    #### 2. Preprocessing
    '''감성 어휘 사전 : negative / positive
       뉴스 데이터 : samsung / hyundai / lg / sk
       주식 데이터 : stock_samsung / stock_hyundai / stock_lg / stock_sk
       공휴일 데이터 : holidays'''
    
    
    ### 1) 뉴스 데이터 날짜 조정
    
    ## 1-1)업로드 시각 컬럼 추가
    df['time'] = df['datetime'].str[-2:]
    
    ## 1-2) 전일 15시 ~ 금일 15시로 날짜 조정
    after_market = ['15', '16', '17', '18', '19', '20', '21', '22', '23']

    for j in range(len(df['time'])):
        if df['time'][j] in after_market:
            df['date'][j] += datetime.timedelta(1)
        else:
            pass
    
    ## 1-3) 텍스트 전처리
    # \n, \t, \r 제거
    df['text'] = df['text'].str.replace('[\n|\t|\r]', '')
    df['text'] = df['text'].str.replace('\[|\]', '')
    df['text'] = df['text'].str.replace('\<.*\>', '')
    df['text'] = df['text'].str.replace('\\', '')
    df['text'] = df['text'].str.replace('이미지', '')


    ### 2) 주말 및 공휴일 제외
    
    ## 2-1) 주말 및 공휴일만 추출
    market_closed =holidays[holidays['holiday']=="O"].reset_index(drop=True)
    
    ## 2-3) 휴장일 List 생성
    market_closed_list = list(market_closed['date'])
    
    ## 2-4) iteration limit 조정
    limit_number = 15000
    sys.setrecursionlimit(limit_number)
    
    
    while len(df[df['date'].isin(market_closed_list)]['date']) != 0:
        for hoil in df[df['date'].isin(market_closed_list)]['date'].index:
            df['date'][hoil] += datetime.timedelta(1)
    
    ### 3) Tokenization 컬럼 추가
    
    df['Tokenization'] = 0
    rows = df.shape[0]
    for row in range(rows):
        hangeul = re.compile('[^ ㄱ-ㅣ 가-힣]')                                    # 정규 표현식 → 한글 추출 규칙 : 띄어쓰기(1개)를 포함한 한글
        result = hangeul.sub('', df['text'][row])   # 위에 설정한 hangeul 규칙을 text에 적용
        okt = Okt()                                                               # 형태소 추출
        nouns = okt.nouns(df['text'][row])
        nouns = [x for x in nouns if len(x) > 1]                                  # 한 글자 키워드 제거
        nouns = [x for x in nouns if x not in stopwords]                          # 불용어 제거
        
        corpus = " ".join(nouns)                                                  # List를 String으로 변환
        df['Tokenization'][row] = corpus


    # 점수 산출

    df['Positive_Score'] = 0
    df['Negative_Score'] = 0
    df['Ratio'] = 0.1
    df['Pred'] = 0
    df['NSI'] = 0.1
    
    for score in range(len(df)):
        pos_score = 0 ; neg_score = 0
        
        for token in range(len(df['Tokenization'][score].split())):
            if df['Tokenization'][score].split()[token] in positive:
                pos_score += 1
            elif df['Tokenization'][score].split()[token] in negative:
                neg_score += 1
            else:
                pass
        
        df['Positive_Score'][score] = pos_score
        df['Negative_Score'][score] = neg_score
        
        # 긍정과 부정의 비율
        if (pos_score==0) and (neg_score==0):
            df['Ratio'][score] = 0.5   # 둘 다 0일 경우에는 긍정으로 가정
        else:
            df['Ratio'][score] = pos_score / (pos_score + neg_score)
        
        # 예측 결과
        if df['Ratio'][score]>=0.5:
            df['Pred'][score] = 1
        else:
            df['Pred'][score] = -1
        
        # 뉴스심리지수(NSI) 계산
        if (pos_score==0) and (neg_score==0):
            df['NSI'][score] = 101
        else:
            df['NSI'][score] = (pos_score - neg_score) / (pos_score + neg_score) * 100 + 100
    
    return df

### 매일경제

In [56]:
# mail_df = news_df[(news_df['news'] == '매일경제') & (news_df['st_cd'] == 5930)]

In [57]:
mail_df = news_db('maeil', '051910')

In [58]:
mail_df.head()

,st_n,st_cd,news,datetime,title,url,text,date,time,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
0,LG화학,051910,매일경제,2021091214,"""10만원에 페이스북 아마존 애플 넷플릭스 구글 다 살 수 있다?""",http://news.mk.co.kr/newsRead.php?no=878988&ye...,"◆ 고가주식 쪼개서 매매 ◆ 아마존, 알파벳, LG생활건강 등 주당 100만원이 훌...",2021-09-13,14,고가 주식 매매 아마존 알파벳 생활 건강 훌쩍 고가 우량 커피 투자 길이 린다 금융...,0,1,0.0,-1,0.0
1,LG화학,051910,매일경제,2021091218,10만원으로 FAANG 분산투자 가능…지갑 얇은 MZ세대 솔깃,http://news.mk.co.kr/newsRead.php?no=879797&ye...,◆ 고가주식 쪼개서 매매 ◆ 중소기업에 근무하는 김아현 씨(31·가명)는 종잣돈 ...,2021-09-13,18,고가 주식 매매 중소기업 근무 김아현 가명 종잣돈 올해 주식 투자 전셋집 사방 팔방...,0,1,0.0,-1,0.0
2,LG화학,051910,매일경제,2021091404,"LG, AI인재 확보 위해 3년간 2000억 투자",http://news.mk.co.kr/newsRead.php?no=884224&ye...,◆ 인재경영 상생 ◆LG전자 사내 소프트웨어 전문가 교육과정 메타버스 수료식. 사진...,2021-09-14,04,인재 경영 상생 사내 소프트웨어 전문가 교육과정 메타 버스 수료식 사진 제공 구광모...,9,0,1.0,1,200.0
3,LG화학,051910,매일경제,2021091404,[인재경영 상생] 경영 불확실성 여전하지만…미래 '기둥' 영입은 계속된다,http://news.mk.co.kr/newsRead.php?no=884228&ye...,"사진 출처 = 게티뱅크 코로나19가 델타, 뮤 등 다양한 변이 양상을 보이며 경제 ...",2021-09-14,04,사진 출처 게티 뱅크 코로나 델타 양상 보이 관련 불확실 지속 반면 노동조합 개정 ...,13,0,1.0,1,200.0
4,LG화학,051910,매일경제,2021091410,"LG화학, 美 메이저 곡물기업과 합작공장 설립…""100% 바이오 플라스틱 만든다""",http://news.mk.co.kr/newsRead.php?no=885210&ye...,LG화학이 글로벌 4대 메이저 곡물 가공 기업인 미국 아처 대니얼스 미들랜드(ADM...,2021-09-14,10,화학 글로벌 메이저 곡물 가공 기업인 미국 대니얼 미들랜드 합작 공장 설립 옥수수 ...,4,0,1.0,1,200.0


In [59]:
mail_ratio = mail_df[mail_df['date'].isin(list(total_table.date.values))][['date','Ratio']]
mail_nsi = mail_df[mail_df['date'].isin(list(total_table.date.values))][['date','NSI']]

In [60]:
mail_ratio = mail_ratio.groupby(['date']).mean().reset_index()
mail_nsi = mail_nsi.groupby(['date']).mean().reset_index()

In [61]:
mail_ratio.columns = ['date', 'mail_news']
mail_nsi.columns = ['date', 'mail_news_nsi']

In [62]:
total_table = pd.merge(total_table, mail_ratio, on='date', how='left')
total_table = pd.merge(total_table, mail_nsi, on='date', how='left')

In [63]:
total_table = total_table[['code', 'date', 'lstm', 'arima', 'fbprophet', 'RL', 'mail_news', 'mail_news_nsi', 'close', 'change']]

In [64]:
total_table

,code,date,lstm,arima,fbprophet,RL,mail_news,mail_news_nsi,close,change
0,051910,2021-09-15,733916.571140,729389.675560,750428.645077,1,0.550000,110.000000,718000,-0.021798
1,051910,2021-09-16,730745.079160,711531.412953,755533.388291,0,0.316667,63.333333,716000,-0.002786
2,051910,2021-09-17,727474.198222,705344.846383,755657.104610,-1,0.633333,126.866667,701000,-0.020950
3,051910,2021-09-23,723239.910305,689864.156645,748824.721482,-1,0.533333,106.666667,760000,0.084165
4,051910,2021-09-24,721332.248360,772048.242011,747809.020471,-1,0.200000,40.000000,761000,0.001316
5,051910,2021-09-27,721468.415320,775235.263126,738985.498854,-1,0.833333,166.666667,770000,0.011827
6,051910,2021-09-28,725266.111761,794974.525572,736767.834210,-1,0.317460,63.492063,776000,0.007792
7,051910,2021-09-29,730308.284700,781150.824165,730421.099476,-1,0.000000,0.000000,767000,-0.011598
8,051910,2021-09-30,732722.755849,769613.268728,733394.311545,1,0.416667,83.666667,776000,0.011734
9,051910,2021-10-01,736871.808171,777926.111086,731385.012952,-1,0.742690,148.538012,770000,-0.007732


### 아시아

In [65]:
# asia_df = news_df[(news_df['news'] == '아시아경제') & (news_df['st_cd'] == 5930)]

In [66]:
asia_df = news_db('asia', '051910')

In [67]:
asia_df.tail()

,st_n,st_cd,news,datetime,title,url,text,date,time,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
50,LG화학,051910,아시아경제,2021092914,美 증시 충격 여파…오후에도 코스피·코스닥 하락 지속,https://view.asiae.co.kr/article/2021092914013...,썝蹂몃낫湲 븘씠肄아시아경제 이민지 기자 미국 증시 급락 여파에 코스피와 코스닥지수가...,2021-09-29,14,몃낫 븘씠 이민지 기자 미국 증시 급락 여파 코스피 코스닥 지수 오후 하락 어가 오...,0,28,0.000000,-1,0.000000
51,LG화학,051910,아시아경제,2021092916,美 '증시 충격'에 코스피·코스닥 동반 하락 마감,https://view.asiae.co.kr/article/2021092916084...,"美증시 급락 여파… 안전자산 선호 심리 강화반도체 투심 위축, 삼전·SK하이닉스 3...",2021-09-30,16,증시 급락 여파 안전 자산 선호 심리 강화 반도체 투심 위축 삼전 하이닉스 내외 하...,0,55,0.000000,-1,0.000000
52,LG화학,051910,아시아경제,2021100106,"""글로벌 공급망 인권보호 의무 강화… 중소 수출기업도 대비해야""",https://view.asiae.co.kr/article/2021100106004...,전경련 'ESG 공급망 인권 관리 동향과 시사점' 보고서 통해 주장 자료제공=전경련...,2021-10-01,06,전경련 공급망 인권 관리 동향 시사 보고서 통해 주장 자료 제공 전경련 몃낫 븘씠 ...,4,6,0.400000,-1,80.000000
53,LG화학,051910,아시아경제,2021100111,"'고객 가치' 다시 강조한 구광모…""재무지표가 사업 목적 돼선 안돼""",https://view.asiae.co.kr/article/2021100110143...,지난달 30일 LG '사장단 워크샵'서 강조고객 접점 - 미래 준비 방점 두고 현장...,2021-10-01,11,지난달 장단 워크샵 강조 고객 접점 미래 준비 방점 현장 경영 행보 구광모 회장 지...,20,3,0.869565,1,173.913043
54,LG화학,051910,아시아경제,2021100116,코스피 3000선 위협...코스닥도 천스닥 아래로,https://view.asiae.co.kr/article/2021100116073...,코스피 지수가 미국 증시 부진 여파에 하락 출발한 1일 서울 을지로 하나은행 딜링룸...,2021-10-05,16,코스피 지수 미국 증시 부진 여파 하락 출발 서울 을지로 하나은행 딜링룸 딜러 업무...,1,21,0.045455,-1,9.090909


In [68]:
asia_ratio = asia_df[asia_df['date'].isin(list(total_table.date.values))][['date','Ratio']]
asia_nsi = asia_df[asia_df['date'].isin(list(total_table.date.values))][['date','NSI']]

asia_ratio = asia_ratio.groupby(['date']).mean().reset_index()
asia_nsi = asia_nsi.groupby(['date']).mean().reset_index()

asia_ratio.columns = ['date', 'asia_news']
asia_nsi.columns = ['date', 'asia_news_nsi']

In [69]:
total_table = pd.merge(total_table, asia_ratio, on='date', how='left')
total_table = pd.merge(total_table, asia_nsi, on='date', how='left')

In [70]:
# 컬럼 리셋
total_table = total_table[['code', 'date', 'lstm', 'arima', 'fbprophet', 'RL', 'mail_news', 'mail_news_nsi', 'asia_news', 'asia_news_nsi', 'close', 'change']]

In [71]:
total_table

,code,date,lstm,arima,fbprophet,RL,mail_news,mail_news_nsi,asia_news,asia_news_nsi,close,change
0,051910,2021-09-15,733916.571140,729389.675560,750428.645077,1,0.550000,110.000000,0.536033,107.206519,718000,-0.021798
1,051910,2021-09-16,730745.079160,711531.412953,755533.388291,0,0.316667,63.333333,0.783862,156.772487,716000,-0.002786
2,051910,2021-09-17,727474.198222,705344.846383,755657.104610,-1,0.633333,126.866667,0.291355,58.271062,701000,-0.020950
3,051910,2021-09-23,723239.910305,689864.156645,748824.721482,-1,0.533333,106.666667,0.548753,109.750513,760000,0.084165
4,051910,2021-09-24,721332.248360,772048.242011,747809.020471,-1,0.200000,40.000000,0.420387,84.077381,761000,0.001316
5,051910,2021-09-27,721468.415320,775235.263126,738985.498854,-1,0.833333,166.666667,0.521825,104.365079,770000,0.011827
6,051910,2021-09-28,725266.111761,794974.525572,736767.834210,-1,0.317460,63.492063,0.181818,36.363636,776000,0.007792
7,051910,2021-09-29,730308.284700,781150.824165,730421.099476,-1,0.000000,0.000000,0.000000,0.000000,767000,-0.011598
8,051910,2021-09-30,732722.755849,769613.268728,733394.311545,1,0.416667,83.666667,0.000000,0.000000,776000,0.011734
9,051910,2021-10-01,736871.808171,777926.111086,731385.012952,-1,0.742690,148.538012,0.634783,126.956522,770000,-0.007732


In [72]:
# total_table.to_csv('total_table.csv', index=False)

## youtube 긍부정 결과
- ['한국경제TV', '슈카월드', '삼프로TV_경제의신과함께']

In [73]:
# # 이전 라벨링 데이터
# youtube_df = pd.read_csv('/content/drive/My Drive/Final PJT - 업빛투/감성분석/data/youtube_naver_labeling.csv')

In [74]:
# youtube_df.ch_nm.unique()

### youtube 데이터 전체 로드

In [75]:
def youtube_db():

    com_list = {'하이닉스' : '000660',
            '현대차' : '005380',
            '삼성전자' : '005930',
            'LG화학' : '051910',
            '셀트리온' : '068270'}
    media_list = {'youtube_한국경제_':'hk_',
                'youtube_삼프로_':'sampro_',
                'youtube_슈카_':'suka_'}
    df_list = []
    for n,m in media_list.items():
        for i,j in com_list.items():

            db = pymysql.connect(
                user='root', 
                passwd='1234', 
                host='3.35.70.166', 
                db='proj', 
                charset = 'utf8'
            )

            cursor = db.cursor(pymysql.cursors.DictCursor)
            sql = "select * from youtube_"+ m + j +" where length(date) = 10"
            
            cursor.execute(sql)

            result = cursor.fetchall()

            # 데이터 프레임으로 변경
            globals() ['df_'+ m + j ] = pd.DataFrame(result)
            print('df_'+ m + j)
            df_list.append(globals() ['df_'+ m + j ])
            
            db.close()
    
    # print(df_list)
    df_total = pd.concat(df_list)
    
    df_total.rename(columns={'date': 'datetime'}, inplace=True)
    # DataFrame 형태를 통일하기 위해 date 컬럼 추가
    df_total['date'] = df_total['datetime'].str[0:4] + '-' + df_total['datetime'].str[4:6] + '-' + df_total['datetime'].str[6:8]
    df_total['date'] = pd.to_datetime(df_total['date'])

    # 시간순으로 정렬
    df_total.reset_index(inplace=True, drop=True)

    ### 4) Holidays
    db = pymysql.connect(user='root',
                         passwd='1234', 
                         host='3.35.70.166', 
                         db='proj',
                         charset='utf8')

    cursor = db.cursor(pymysql.cursors.DictCursor)

    # 4-1) 주말 및 공휴일 데이터
    sql = "select * from holidays"
    cursor.execute(sql)
    result = cursor.fetchall()

    holidays = pd.DataFrame(result)
    db.close()

    holidays['date'] = pd.to_datetime(holidays['date'])

    ## 2-1) 주말 및 공휴일만 추출
    market_closed =holidays[holidays['holiday']=="O"].reset_index(drop=True)
    
    ## 2-3) 휴장일 List 생성
    market_closed_list = list(market_closed['date'])
    
    ## 2-4) iteration limit 조정
    limit_number = 15000
    sys.setrecursionlimit(limit_number)
    
    
    while len(df_total[df_total['date'].isin(market_closed_list)]['date']) != 0:
        for hoil in df_total[df_total['date'].isin(market_closed_list)]['date'].index:
            df_total['date'][hoil] += datetime.timedelta(1)

    # 간단한 전처리
    df_total['text'] = df_total['text'].str.replace('\[|\]','')
    df_total['text'] = df_total['text'].str.replace('[a-z]','')
    df_total['text'] = df_total['text'].str.replace('[A-Z]','')
    df_total['text'] = df_total['text'].str.replace('으','')
    df_total['text'] = df_total['text'].str.replace('음악','')
    df_total['text'] = df_total['text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
    df_total['text'] = df_total['text'].str.replace('^ +', "")

    return df_total

In [76]:
df_total = youtube_db()

df_hk_000660
df_hk_005380
df_hk_005930
df_hk_051910
df_hk_068270
df_sampro_000660
df_sampro_005380
df_sampro_005930
df_sampro_051910
df_sampro_068270
df_suka_000660
df_suka_005380
df_suka_005930
df_suka_051910
df_suka_068270


### 라벨링 패키지 및 모델 로딩

In [77]:
# 라벨 훈련 데이터
X_train = []
aa = pd.read_csv('/content/drive/My Drive/Final PJT - 업빛투/감성분석/youtube/train_label.csv').drop('Unnamed: 0',1)
aa.x = aa.x.str.replace('\[|\]','')
aa.x = aa.x.str.replace("'","")
aa.x = aa.x.str.replace(" ","")

for i in aa.x:
    temp = i.split(',')
    X_train.append(temp)

In [78]:
len(X_train)

145393

In [79]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
tokenizer = Tokenizer(19196)
tokenizer.fit_on_texts(X_train)
url = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/Sentiment_Analysis/data/stopwords_ver1.txt'
stopwords = list(pd.read_csv(url, header=None)[0])

In [80]:
max_len = 200

In [81]:
# loaded_model = load_model('./best_model.h5')

In [82]:
loaded_model = load_model('/content/drive/My Drive/Final PJT - 업빛투/감성분석/youtube/best_model.h5')

In [83]:
def sentiment_predict(new_sentence):
    new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(loaded_model.predict(pad_new)) # 예측

    if (score > 0.5):
        # print("{:.2f}% 확률로 긍정 리뷰\n".format(score * 100))
        return score, 1
    elif (score == 0.5):
        return score, 0
    else:
        # print("{:.2f}% 확률로 부정 리뷰\n".format((1 - score) * 100))
        return score, -1

In [84]:
# sentiment_predict('꿀잼') 연습

### 삼프로

In [85]:
sampro_df = df_total[(df_total.ch_nm == '삼프로TV_경제의신과함께') & (df_total.st_cd == '051910')]

In [86]:
sampro_df = sampro_df[sampro_df.date.isin(total_table.date)]

In [87]:
s_text = sampro_df.text.values

In [88]:
scores = [] # 긍정 확률
for i in s_text:
    score, _ = sentiment_predict(i)
    scores.append(score)

In [89]:
len(sampro_df),len(scores)

(14, 14)

In [90]:
sampro_df['sampro_youtube'] = np.array(scores).reshape(-1,1)
sampro_score = sampro_df.groupby(['date']).mean().reset_index()[['date','sampro_youtube']]

In [91]:
total_table = pd.merge(total_table, sampro_score, on='date',how='left')

In [92]:
total_table

,code,date,lstm,arima,fbprophet,RL,mail_news,mail_news_nsi,asia_news,asia_news_nsi,close,change,sampro_youtube
0,051910,2021-09-15,733916.571140,729389.675560,750428.645077,1,0.550000,110.000000,0.536033,107.206519,718000,-0.021798,0.931033
1,051910,2021-09-16,730745.079160,711531.412953,755533.388291,0,0.316667,63.333333,0.783862,156.772487,716000,-0.002786,NaN
2,051910,2021-09-17,727474.198222,705344.846383,755657.104610,-1,0.633333,126.866667,0.291355,58.271062,701000,-0.020950,NaN
3,051910,2021-09-23,723239.910305,689864.156645,748824.721482,-1,0.533333,106.666667,0.548753,109.750513,760000,0.084165,0.036642
4,051910,2021-09-24,721332.248360,772048.242011,747809.020471,-1,0.200000,40.000000,0.420387,84.077381,761000,0.001316,0.061370
5,051910,2021-09-27,721468.415320,775235.263126,738985.498854,-1,0.833333,166.666667,0.521825,104.365079,770000,0.011827,0.150472
6,051910,2021-09-28,725266.111761,794974.525572,736767.834210,-1,0.317460,63.492063,0.181818,36.363636,776000,0.007792,0.278171
7,051910,2021-09-29,730308.284700,781150.824165,730421.099476,-1,0.000000,0.000000,0.000000,0.000000,767000,-0.011598,0.114135
8,051910,2021-09-30,732722.755849,769613.268728,733394.311545,1,0.416667,83.666667,0.000000,0.000000,776000,0.011734,0.140898
9,051910,2021-10-01,736871.808171,777926.111086,731385.012952,-1,0.742690,148.538012,0.634783,126.956522,770000,-0.007732,0.367717


### 한국경제

In [93]:
hk_df = df_total[(df_total.ch_nm == '한국경제TV') & (df_total.st_cd == '051910')]

In [94]:
hk_df = hk_df[hk_df.date.isin(total_table.date)]

In [95]:
hk_text = hk_df.text.values

scores = [] # 긍정 확률
for i in hk_text:
    score, _ = sentiment_predict(i)
    scores.append(score)

In [96]:
len(scores), len(hk_df)

(8, 8)

In [97]:
hk_df['hk_youtube'] = np.array(scores).reshape(-1,1)
hk_score = hk_df.groupby(['date']).mean().reset_index()[['date','hk_youtube']]

total_table = pd.merge(total_table, hk_score, on='date',how='left')

### 슈카월드

In [98]:
su_df = df_total[(df_total.ch_nm == '슈카월드') & (df_total.st_cd == '051910')]

su_df = su_df[su_df.date.isin(total_table.date)]

su_text = su_df.text.values

scores = [] # 긍정 확률
for i in su_text:
    score, _ = sentiment_predict(i)
    scores.append(score)

In [99]:
len(su_df),len(scores)

(1, 1)

In [100]:
su_df['suka_youtube'] = np.array(scores).reshape(-1,1)
suka_score = su_df.groupby(['date']).mean().reset_index()[['date','suka_youtube']]

total_table = pd.merge(total_table, suka_score, on='date',how='left')

In [101]:
total_table.columns

Index(['code', 'date', 'lstm', 'arima', 'fbprophet', 'RL', 'mail_news',
       'mail_news_nsi', 'asia_news', 'asia_news_nsi', 'close', 'change',
       'sampro_youtube', 'hk_youtube', 'suka_youtube'],
      dtype='object')

In [102]:
total_table = total_table[['code', 'date', 'lstm', 'arima', 'fbprophet', 'RL', 'mail_news',
       'mail_news_nsi', 'asia_news', 'asia_news_nsi',
       'sampro_youtube', 'hk_youtube', 'suka_youtube', 'close', 'change']]

# 최종결과

In [103]:
total_table

,code,date,lstm,arima,fbprophet,RL,mail_news,mail_news_nsi,asia_news,asia_news_nsi,sampro_youtube,hk_youtube,suka_youtube,close,change
0,051910,2021-09-15,733916.571140,729389.675560,750428.645077,1,0.550000,110.000000,0.536033,107.206519,0.931033,NaN,0.50873,718000,-0.021798
1,051910,2021-09-16,730745.079160,711531.412953,755533.388291,0,0.316667,63.333333,0.783862,156.772487,NaN,0.789423,NaN,716000,-0.002786
2,051910,2021-09-17,727474.198222,705344.846383,755657.104610,-1,0.633333,126.866667,0.291355,58.271062,NaN,NaN,NaN,701000,-0.020950
3,051910,2021-09-23,723239.910305,689864.156645,748824.721482,-1,0.533333,106.666667,0.548753,109.750513,0.036642,0.763710,NaN,760000,0.084165
4,051910,2021-09-24,721332.248360,772048.242011,747809.020471,-1,0.200000,40.000000,0.420387,84.077381,0.061370,NaN,NaN,761000,0.001316
5,051910,2021-09-27,721468.415320,775235.263126,738985.498854,-1,0.833333,166.666667,0.521825,104.365079,0.150472,0.343618,NaN,770000,0.011827
6,051910,2021-09-28,725266.111761,794974.525572,736767.834210,-1,0.317460,63.492063,0.181818,36.363636,0.278171,NaN,NaN,776000,0.007792
7,051910,2021-09-29,730308.284700,781150.824165,730421.099476,-1,0.000000,0.000000,0.000000,0.000000,0.114135,0.931219,NaN,767000,-0.011598
8,051910,2021-09-30,732722.755849,769613.268728,733394.311545,1,0.416667,83.666667,0.000000,0.000000,0.140898,NaN,NaN,776000,0.011734
9,051910,2021-10-01,736871.808171,777926.111086,731385.012952,-1,0.742690,148.538012,0.634783,126.956522,0.367717,0.965788,NaN,770000,-0.007732


In [111]:
while total_table.iloc[-1:].isnull().sum().sum() != 0:
    if total_table.iloc[:1].isnull().sum().sum() != 0:
      for i in range(1, len(total_table)):
        total_table = total_table.fillna(method='pad')
      break
    total_table = total_table.fillna(method='pad')

In [113]:
total_table

,code,date,lstm,arima,fbprophet,RL,mail_news,mail_news_nsi,asia_news,asia_news_nsi,sampro_youtube,hk_youtube,suka_youtube,close,change
0,51910,2021-09-15,733916.571140,729389.675560,750428.645077,1,0.550000,110.000000,0.536033,107.206519,0.931033,NaN,0.50873,718000,-0.021798
1,51910,2021-09-16,730745.079160,711531.412953,755533.388291,0,0.316667,63.333333,0.783862,156.772487,0.931033,0.789423,0.50873,716000,-0.002786
2,51910,2021-09-17,727474.198222,705344.846383,755657.104610,-1,0.633333,126.866667,0.291355,58.271062,0.931033,0.789423,0.50873,701000,-0.020950
3,51910,2021-09-23,723239.910305,689864.156645,748824.721482,-1,0.533333,106.666667,0.548753,109.750513,0.036642,0.763710,0.50873,760000,0.084165
4,51910,2021-09-24,721332.248360,772048.242011,747809.020471,-1,0.200000,40.000000,0.420387,84.077381,0.061370,0.763710,0.50873,761000,0.001316
5,51910,2021-09-27,721468.415320,775235.263126,738985.498854,-1,0.833333,166.666667,0.521825,104.365079,0.150472,0.343618,0.50873,770000,0.011827
6,51910,2021-09-28,725266.111761,794974.525572,736767.834210,-1,0.317460,63.492063,0.181818,36.363636,0.278171,0.343618,0.50873,776000,0.007792
7,51910,2021-09-29,730308.284700,781150.824165,730421.099476,-1,0.000000,0.000000,0.000000,0.000000,0.114135,0.931219,0.50873,767000,-0.011598
8,51910,2021-09-30,732722.755849,769613.268728,733394.311545,1,0.416667,83.666667,0.000000,0.000000,0.140898,0.931219,0.50873,776000,0.011734
9,51910,2021-10-01,736871.808171,777926.111086,731385.012952,-1,0.742690,148.538012,0.634783,126.956522,0.367717,0.965788,0.50873,770000,-0.007732


In [114]:
total_table.to_csv('./total_table.csv', index=False)

In [115]:
total_table = pd.read_csv('./total_table.csv')

In [116]:
total_table_ud = np.round(total_table,2)


for i in range(len(total_table_ud)-1,0,-1):
  if total_table_ud['lstm'].iloc[i] > total_table_ud['lstm'].iloc[i-1]:
    total_table_ud['lstm'].iloc[i] = 1
  else:
    total_table_ud['lstm'].iloc[i] = -1
  
  if total_table_ud['arima'].iloc[i] > total_table_ud['close'].iloc[i-1]:
    total_table_ud['arima'].iloc[i] = 1
  else:
    total_table_ud['arima'].iloc[i] = -1
  
  if total_table_ud['fbprophet'].iloc[i] > total_table_ud['fbprophet'].iloc[i-1]:
    total_table_ud['fbprophet'].iloc[i] = 1
  else:
    total_table_ud['fbprophet'].iloc[i] = -1
  
total_table_ud['mail_news'] = np.where(total_table_ud['mail_news']>0.5,1,
                                       np.where(total_table_ud['mail_news']<0.5,-1,0))

total_table_ud['mail_news_nsi'] = np.where(total_table_ud['mail_news_nsi']>100,1,
                                       np.where(total_table_ud['mail_news_nsi']<100,-1,0))

total_table_ud['asia_news'] = np.where(total_table_ud['asia_news']>0.5,1,
                                       np.where(total_table_ud['asia_news']<0.5,-1,0))

total_table_ud['asia_news_nsi'] = np.where(total_table_ud['asia_news_nsi']>100,1,
                                       np.where(total_table_ud['asia_news_nsi']<100,-1,0))

total_table_ud['sampro_youtube'] = np.where(total_table_ud['sampro_youtube']>0.5,1,
                                       np.where(total_table_ud['sampro_youtube']<0.5,-1,0))

total_table_ud['hk_youtube'] = np.where(total_table_ud['hk_youtube']>0.5,1,
                                       np.where(total_table_ud['hk_youtube']<0.5,-1,0))

total_table_ud['suka_youtube'] = np.where(total_table_ud['suka_youtube']>0.5,1,
                                       np.where(total_table_ud['suka_youtube']<0.5,-1,0))

total_table_ud['change'] = np.where(total_table_ud['change']>0,1,
                                       np.where(total_table_ud['change']<0,-1,0))
total_table_ud.drop('close',1,inplace=True)

In [117]:
total_table_ud

,code,date,lstm,arima,fbprophet,RL,mail_news,mail_news_nsi,asia_news,asia_news_nsi,sampro_youtube,hk_youtube,suka_youtube,change
0,51910,2021-09-15,733916.57,729389.68,750428.65,1,1,1,1,1,1,0,1,-1
1,51910,2021-09-16,-1.00,-1.00,1.00,0,-1,-1,1,1,1,1,1,0
2,51910,2021-09-17,-1.00,-1.00,1.00,-1,1,1,-1,-1,1,1,1,-1
3,51910,2021-09-23,-1.00,-1.00,-1.00,-1,1,1,1,1,-1,1,1,1
4,51910,2021-09-24,-1.00,1.00,-1.00,-1,-1,-1,-1,-1,-1,1,1,0
5,51910,2021-09-27,1.00,1.00,-1.00,-1,1,1,1,1,-1,-1,1,1
6,51910,2021-09-28,1.00,1.00,-1.00,-1,-1,-1,-1,-1,-1,-1,1,1
7,51910,2021-09-29,1.00,1.00,-1.00,-1,-1,-1,-1,-1,-1,1,1,-1
8,51910,2021-09-30,1.00,1.00,1.00,1,-1,-1,-1,-1,-1,1,1,1
9,51910,2021-10-01,1.00,1.00,-1.00,-1,1,1,1,1,-1,1,1,-1


In [119]:
total_table_ud.to_csv('./lg_table_ud.csv', index=False)